# 4. Linear regression

In [6]:
from random import seed, randrange

# Importing our own functions
from functions import *

### Define functions
* Mean, variance, and covariance
* Simple linear regression
* Multiple linear regression using stochastic gradient descent

In [7]:
# Calculate the mean value of a list of numbers
def mean(values):
    return sum(values) / float(len(values))

# Calculate variance of a lsit of numbers, provided mean
def variance(values, mean):
    return sum([(x - mean)**2 for x in values])

# Calculate covariance between x and y
def covariance(x, x_mean, y, y_mean):
    covar = 0.0
    for i in range(len(x)):
        covar += (x[i] - x_mean) * (y[i] - y_mean)
    return covar

# Calculate coefficients
def coefficients(dataset):
    # Assumes x is first column, y second - creating a list for each
    x = [row[0] for row in dataset]
    y = [row[1] for row in dataset]
    x_mean, y_mean = mean(x), mean(y)
    # Calculate simple linear regression coefficients
    b1 = covariance(x, x_mean, y, y_mean) / variance(x, x_mean)
    b0 = y_mean - b1 * x_mean
    # Return list of coefficients
    return [b0, b1]

# Simple linear regression, assuming train/test split
def simple_linear_regression(train, test):
    predictions = list()
    # Calculate regression coefficients
    b0, b1 = coefficients(train)
    # Test data used only to count iterations; parameter can be removed
    # Include anwyway since most other algos require both test and train sets
    for row in test:
        # For each observation, predict y^ given x and coefficients
        yhat = b0 + b1 * row[0]
        predictions.append(yhat)
    return predictions

# Make a prediction with provided coefficients
# Returns yhat, i.e. predicted y-value for the provided row
def predict_linear(row, coefficients):
    # First coefficient in is always the intercept B0 ("bias")
    yhat = coefficients[0]
    # Iterate over remaining columns in row, multiplying Xi * Bi
    for i in range(len(row)-1):
        # Sum for prediction
        yhat += coefficients[i + 1] * row[i]
    return yhat

# Estimate linear regression coefficients using stochastic gradient descent
# Returns list of coefficients, with intercept at first index
# Batch size b = 1 means stochastic gradient descent, b > 1 means (mini) batch gradient descent
def coefficients_linear_sgd(train, l_rate, n_epoch, b = 1):
    # Start with empty coefficients
    coef = [0.0 for col in range(len(train[0]))]
    # Iterate over epochs, performing SGD based on batch size for each
    for epoch in range(n_epoch):
        # Sum total error to track overall epoch performance
        sum_error = 0
        # Counter to track mini batch
        i = 0
        # Counter to track overall row, to capture mid-batch end of dataset
        j = 0
        # SUM(h - y) * Xi over coefficients i, to multiply full batch by learning rate
        adjustments = [0 for col in range(len(train[0]))]
        # Iterate over all rows of data
        for row in train:
            # Increment batch index and dataset counters
            i += 1
            j += 1
            # Predict y for this row with latest coefficients
            yhat = predict_linear(row, coef)
            # Find prediction error, assuming actual y is final column; i.e. (h - y)
            error = yhat - row[-1]
            # Squaring for absolute error, to track epoch performance
            sum_error += error**2
            # Add intercept (bias) error, since has no Xi
            adjustments[0] += error
            # Iterate over all coefficients, except the first (i.e. intercept)
            for k in range(len(row)-1):
                # Add to batch adjustment, i.e. (h - y) * Xi
                # row column j belongs to coefficient j + 1, since j = 1 is intercept
                adjustments[k + 1] += error * row[k]
            # Check whether this row is last in batch, or in dataset
            if i == b or j == len(train):
                # Iterate over all coefficients
                for k in range(len(adjustments)):
                    # Adjust coefficient based on batch sum gradient and learning rate
                    # i = b except if at end of dataset and in mid-batch
                    # "Averaging" adjustment of each row in mini batch
                    coef[k] = coef[k] - (1/i) * l_rate * adjustments[k]
                # Reset adjustments to 0 for next batch
                adjustments = [0 for col in range(len(train[0]))]
                # Reset batch counter
                i = 0
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch+1, l_rate, sum_error))
    # Return the final set of coefficients
    return coef

# Linear Regression Algorithm With Stochastic Gradient Descent
# Returns the list of predictions for each dataset row
# Parameters: learning rate, number of epochs (iterations), mini-batch size b
def linear_regression_sgd(train, test, l_rate, n_epoch, b_size = 1):
    predictions = list()
    # Get coefficients based on SGD, learning rate and epochs
    coef = coefficients_linear_sgd(train, l_rate, n_epoch, b_size)
    # Get predictions for provided dataset using estimated coefficients
    for row in test:
        yhat = predict_linear(row, coef)
        predictions.append(yhat)
    return predictions


### Testing simple linear regression

In [8]:
# Test calculate mean, variance, and covariance
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
x = [row[0] for row in dataset]
y = [row[1] for row in dataset]
mean_x, mean_y = mean(x), mean(y)
var_x, var_y = variance(x, mean_x), variance(y, mean_y)
covar = covariance(x, mean_x, y, mean_y)
print('x stats: mean=%.3f variance=%.3f' % (mean_x, var_x))
print('y stats: mean=%.3f variance=%.3f' % (mean_y, var_y))
print('Covariance: %.3f' % (covar))

# Test calculating coefficients and finding predictions
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
# Generally, nullify test set to avoid accidental cheating
# (Our simple linear regression only uses the test set to iterate over rows, however)
test_set = list()
for row in dataset:
    row_copy = list(row)
    row_copy[-1] = None
    test_set.append(row_copy)
    
print('\nContrived dataset:', dataset)
b0, b1 = coefficients(dataset)
print('Coefficients: B0=%.3f, B1=%.3f' % (b0, b1))
predicted = simple_linear_regression(dataset, test_set)
print('Predictions:', simple_linear_regression(dataset, test_set))
actual = [row[-1] for row in dataset]
print('RMSE:', rmse_metric(actual, predicted))

# Testing simple linear regression and RMSE evaluation on contrived dataset
print('\nTesting contrived dataset, using default 0.6 split:')
seed(1)
rmse = evaluate_algorithm(dataset, simple_linear_regression, 0.6, rmse_metric)
print('RMSE: %.3f' % (rmse))
print('Note: Expect higher RMSE since here trained on 3 observations and tested on 2')

# Testing simple linear regression and RMSE evaluation on Swedish Auto Insurance
print('\nTesting Swedish Auto Insurance:', )
# Load dataset and convert columns to floats
dataset = load_csv('data/insurance.csv')
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# Run and evaluate simple linear regression algorithm using dynamic harness
rmse = evaluate_algorithm(dataset, simple_linear_regression, 0.6, rmse_metric)
print('RMSE: %.3f' % (rmse))

x stats: mean=3.000 variance=10.000
y stats: mean=2.800 variance=8.800
Covariance: 8.000

Contrived dataset: [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
Coefficients: B0=0.400, B1=0.800
Predictions: [1.1999999999999995, 1.9999999999999996, 3.5999999999999996, 2.8, 4.3999999999999995]
RMSE: 0.692820323027551

Testing contrived dataset, using default 0.6 split:
RMSE: 1.061
Note: Expect higher RMSE since here trained on 3 observations and tested on 2

Testing Swedish Auto Insurance:
Loaded data file data/insurance.csv with 63 rows and 2 columns.
RMSE: 41.321


### Testing multiple linear regression

In [9]:
# Contrived dataset for testing
dataset = [[1, 1, 2], [2, 3, 1], [4, 3, 4], [3, 2, 2], [5, 5, 4]]  # x, y (actual)

# Test prediction with provided coefficients
coef = [0.4, 0.8, 0.6]   # y = 0.4 + 0.8x
for row in dataset:
    yhat = predict_linear(row, coef)
    print("Expected=%.1f, Predicted=%.1f" % (row[-1], yhat))

# Test determining coefficients using stochastic gradient descent
print()
# Higher learning rate means greater leaps, but less able to fine-tune
l_rate = 0.001
# If learning rate is lower, we need more epochs to achieve higher accuracy
n_epoch = 100
coef = coefficients_linear_sgd(dataset, l_rate, n_epoch, 1)
print("\nIntercept:    ", coef[0])
print("Coefficient 1:", coef[1])
print("Coefficient 2:", coef[2])

# Linear Regression With Stochastic Gradient Descent for Wine Quality
seed(1)
print("\nLinear Regression With Stochastic Gradient Descent for Wine Quality:")

# Load and convert data to float
filename = 'data/winequality-white.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# Normalize, i.e. scale each value to between 0 and 1
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Evaluate linear regression with SGD using dynamic test harness
# 4,898 / 5 = 3916 rows for training and 979 for testing per fold, times five folds
n_folds = 5
l_rate = 0.01
n_epoch = 50
batch_size = 10
# dataset, algorithm, n_folds, split, metric, *args (used to pass l_rate and n_epoch)
scores = evaluate_algorithm(dataset, linear_regression_sgd, 5, rmse_metric, l_rate, n_epoch, batch_size)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

Expected=2.0, Predicted=1.8
Expected=1.0, Predicted=3.8
Expected=4.0, Predicted=5.4
Expected=2.0, Predicted=4.0
Expected=4.0, Predicted=7.4

>epoch=1, lrate=0.001, error=38.455
>epoch=2, lrate=0.001, error=31.701
>epoch=3, lrate=0.001, error=26.254
>epoch=4, lrate=0.001, error=21.861
>epoch=5, lrate=0.001, error=18.318
>epoch=6, lrate=0.001, error=15.460
>epoch=7, lrate=0.001, error=13.154
>epoch=8, lrate=0.001, error=11.294
>epoch=9, lrate=0.001, error=9.793
>epoch=10, lrate=0.001, error=8.582
>epoch=11, lrate=0.001, error=7.604
>epoch=12, lrate=0.001, error=6.815
>epoch=13, lrate=0.001, error=6.177
>epoch=14, lrate=0.001, error=5.662
>epoch=15, lrate=0.001, error=5.246
>epoch=16, lrate=0.001, error=4.910
>epoch=17, lrate=0.001, error=4.637
>epoch=18, lrate=0.001, error=4.417
>epoch=19, lrate=0.001, error=4.238
>epoch=20, lrate=0.001, error=4.093
>epoch=21, lrate=0.001, error=3.976
>epoch=22, lrate=0.001, error=3.880
>epoch=23, lrate=0.001, error=3.802
>epoch=24, lrate=0.001, error=3.

>epoch=11, lrate=0.010, error=2528.876
>epoch=12, lrate=0.010, error=2512.192
>epoch=13, lrate=0.010, error=2496.906
>epoch=14, lrate=0.010, error=2482.823
>epoch=15, lrate=0.010, error=2469.789
>epoch=16, lrate=0.010, error=2457.682
>epoch=17, lrate=0.010, error=2446.401
>epoch=18, lrate=0.010, error=2435.863
>epoch=19, lrate=0.010, error=2425.998
>epoch=20, lrate=0.010, error=2416.747
>epoch=21, lrate=0.010, error=2408.058
>epoch=22, lrate=0.010, error=2399.886
>epoch=23, lrate=0.010, error=2392.191
>epoch=24, lrate=0.010, error=2384.938
>epoch=25, lrate=0.010, error=2378.094
>epoch=26, lrate=0.010, error=2371.631
>epoch=27, lrate=0.010, error=2365.522
>epoch=28, lrate=0.010, error=2359.745
>epoch=29, lrate=0.010, error=2354.276
>epoch=30, lrate=0.010, error=2349.095
>epoch=31, lrate=0.010, error=2344.186
>epoch=32, lrate=0.010, error=2339.529
>epoch=33, lrate=0.010, error=2335.111
>epoch=34, lrate=0.010, error=2330.915
>epoch=35, lrate=0.010, error=2326.930
>epoch=36, lrate=0.010, e